In [1]:
# Import Modules
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Options
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-hueFIJUifG90tn5GAClnT3BlbkFJD8IdTdyNsjuxHviNegtt"

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Load the PDF
b_docs = []

loader = PyPDFLoader("bert.pdf")
b_docs.extend(loader.load())

b_docs[:5]

[Document(metadata={'source': 'bert.pdf', 'page': 0}, page_content='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout }@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT , which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pre-\ntrain deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re-\nsult, the pre-trained BERT model can be ﬁne-\ntuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference, without substantial task-\nspeciﬁc architecture modiﬁcations.\nBERT is conceptually simple

In [3]:
# Create an instance of text-splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Transform raw data into split documents
docs = text_splitter.split_documents(b_docs)
docs[:3]

[Document(metadata={'source': 'bert.pdf', 'page': 0}, page_content='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout }@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT , which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et al., 2018a; Rad-\nford et al., 2018), BERT is designed to pre-\ntrain deep bidirectional representations from\nunlabeled text by jointly conditioning on both\nleft and right context in all layers. As a re-\nsult, the pre-trained BERT model can be ﬁne-\ntuned with just one additional output layer\nto create state-of-the-art models for a wide\nrange of tasks, such as question answering and\nlanguage inference, without substantial task-\nspeciﬁc architecture modiﬁcations.\nBERT is conceptually simple

In [4]:
# Choose text embedding model and transform
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a DB for searching purpose (To retrieve embedding vectors efficiently)
vectorstore = Chroma.from_documents(docs, embeddings)

# Create an instance of retriever
chroma_retriever = vectorstore.as_retriever(
  search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
)

In [5]:
# Prompt Template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context} 
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [6]:
# Model for text generation
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Generate RAG-chain
rag_chain = (
  {"question": RunnablePassthrough(), "context": chroma_retriever}
  | prompt
  | llm
  | StrOutputParser()
)

In [7]:
query = "What are the two main tasks on which BERT is pre-trained on?"
rag_chain.invoke(query)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}